<a href="https://colab.research.google.com/github/Sy31/homework/blob/master/ESM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
ESM-2 650M 蛋白质序列转向量工具
在Google Colab中运行此代码
"""

#@title 1. 安装必要的包
!pip install fair-esm biopython -q
print("✅ 包安装完成")

#@title 2. 导入必要的库
import torch
import esm
import numpy as np
from google.colab import files
import os
import re
from typing import List, Tuple
import gc
from tqdm import tqdm

print("✅ 库导入完成")

#@title 3. 加载ESM-2 650M模型
print("正在加载ESM-2 650M模型...")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用设备: {device}")

# 加载模型
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
model = model.to(device)
model.eval()
batch_converter = alphabet.get_batch_converter()

print("✅ ESM-2 650M模型加载完成")

#@title 4. 定义处理函数
def parse_protein_file(file_path: str) -> List[Tuple[str, str, str]]:
    """
    解析蛋白质文件
    返回: [(index, protein_id, sequence), ...]
    """
    proteins = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line:
                parts = line.split('\t')
                if len(parts) >= 3:
                    idx, protein_id, sequence = parts[0], parts[1], parts[2]
                    # 清理序列，移除可能的非标准字符
                    sequence = re.sub(r'[^ACDEFGHIKLMNPQRSTVWY]', '', sequence.upper())
                    proteins.append((idx, protein_id, sequence))
    return proteins

def get_esm_embeddings(sequences: List[Tuple[str, str]], batch_size: int = 1):
    """
    获取ESM嵌入向量
    """
    embeddings = {}

    # 分批处理以节省内存
    for i in tqdm(range(0, len(sequences), batch_size), desc="生成嵌入向量"):
        batch = sequences[i:i + batch_size]

        # 准备批次数据
        batch_labels, batch_strs, batch_tokens = batch_converter(
            [(label, seq) for label, seq in batch]
        )
        batch_tokens = batch_tokens.to(device)

        # 获取嵌入
        with torch.no_grad():
            results = model(batch_tokens, repr_layers=[33], return_contacts=False)
            token_representations = results["representations"][33]

            # 对每个序列进行平均池化（排除特殊标记）
            for j, (label, seq) in enumerate(batch):
                # 第0个位置是<cls>，最后是<eos>，所以取1:len(seq)+1
                seq_embedding = token_representations[j, 1:len(seq)+1].mean(0)
                embeddings[label] = seq_embedding.cpu().numpy()

        # 清理GPU内存
        del batch_tokens
        torch.cuda.empty_cache()

    return embeddings

def save_embeddings(embeddings: dict, output_dir: str = "protein_embeddings"):
    """
    保存嵌入向量为npy文件
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # 保存每个蛋白质的嵌入
    for protein_id, embedding in embeddings.items():
        file_path = os.path.join(output_dir, f"{protein_id}.npy")
        np.save(file_path, embedding)

    # 也保存所有嵌入在一个文件中
    all_embeddings = np.stack(list(embeddings.values()))
    all_ids = list(embeddings.keys())

    # 保存嵌入矩阵
    np.save(os.path.join(output_dir, "all_embeddings.npy"), all_embeddings)

    # 保存ID列表
    with open(os.path.join(output_dir, "protein_ids.txt"), 'w') as f:
        for pid in all_ids:
            f.write(f"{pid}\n")

    return output_dir

#@title 5. 主处理流程

def process_protein_file(file_path: str, batch_size: int = 1):
    """
    处理上传的蛋白质文件
    """
    print(f"\n📄 处理文件: {file_path}")

    # 1. 解析文件
    print("解析蛋白质序列...")
    proteins = parse_protein_file(file_path)
    print(f"✅ 找到 {len(proteins)} 个蛋白质序列")

    # 2. 准备序列数据
    sequences = [(f"{idx}_{protein_id}", seq) for idx, protein_id, seq in proteins]

    # 显示序列信息
    print("\n序列信息:")
    for idx, protein_id, seq in proteins[:3]:  # 显示前3个
        print(f"  {idx} | {protein_id} | 长度: {len(seq)}")
    if len(proteins) > 3:
        print(f"  ... 还有 {len(proteins)-3} 个序列")

    # 3. 生成嵌入向量
    print(f"\n🔄 生成嵌入向量（批次大小: {batch_size}）...")
    embeddings = get_esm_embeddings(sequences, batch_size)

    # 4. 保存嵌入向量
    print("\n💾 保存嵌入向量...")
    output_dir = save_embeddings(embeddings)

    # 5. 创建压缩文件
    print("\n📦 创建压缩文件...")
    zip_filename = "protein_embeddings.zip"
    !zip -r {zip_filename} {output_dir} -q

    print(f"\n✅ 处理完成！")
    print(f"嵌入向量维度: {list(embeddings.values())[0].shape}")
    print(f"输出文件夹: {output_dir}/")

    return zip_filename, embeddings

#@title 6. 上传文件并处理

print("=" * 50)
print("🚀 ESM-2 650M 蛋白质序列转向量工具")
print("=" * 50)
print("\n请上传你的蛋白质序列文件（txt格式）:")
print("文件格式: ID<tab>蛋白质ID<tab>序列")
print("-" * 50)

# 文件上传
uploaded = files.upload()

if uploaded:
    # 获取上传的文件名
    filename = list(uploaded.keys())[0]
    print(f"\n✅ 文件上传成功: {filename}")

    # 设置批次大小（根据GPU内存调整）
    batch_size = 1  # 对于650M模型，建议使用较小的批次

    try:
        # 处理文件
        zip_file, embeddings = process_protein_file(filename, batch_size)

        # 下载结果
        print("\n📥 准备下载...")
        files.download(zip_file)
        print("✅ 下载准备完成！文件将自动下载。")

        # 显示统计信息
        print("\n📊 统计信息:")
        print(f"  - 处理的蛋白质数量: {len(embeddings)}")
        print(f"  - 嵌入向量维度: {list(embeddings.values())[0].shape[0]}")
        print(f"  - 输出文件:")
        print(f"    • all_embeddings.npy - 所有嵌入向量矩阵")
        print(f"    • protein_ids.txt - 蛋白质ID列表")
        print(f"    • 各个蛋白质的独立.npy文件")

    except Exception as e:
        print(f"\n❌ 处理过程中出现错误: {str(e)}")
        print("请检查文件格式是否正确。")
else:
    print("\n⚠️ 未检测到上传的文件")

#@title 7. （可选）查看嵌入向量示例
if 'embeddings' in locals() and embeddings:
    print("\n📈 嵌入向量示例:")
    first_key = list(embeddings.keys())[0]
    first_embedding = embeddings[first_key]
    print(f"蛋白质ID: {first_key}")
    print(f"嵌入向量形状: {first_embedding.shape}")
    print(f"前10个值: {first_embedding[:10]}")

    # 可视化嵌入向量的分布
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12, 4))

    # 子图1：嵌入向量值的分布
    plt.subplot(1, 2, 1)
    plt.hist(first_embedding, bins=50, edgecolor='black', alpha=0.7)
    plt.title(f'嵌入向量值分布 ({first_key})')
    plt.xlabel('值')
    plt.ylabel('频率')

    # 子图2：前100个维度的值
    plt.subplot(1, 2, 2)
    plt.plot(first_embedding[:100], alpha=0.7)
    plt.title('前100个维度的值')
    plt.xlabel('维度')
    plt.ylabel('值')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

print("\n✨ 完成！如有问题请检查输出信息。")

✅ 包安装完成
✅ 库导入完成
正在加载ESM-2 650M模型...
使用设备: cuda
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/models/esm2_t33_650M_UR50D.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D.pt
Downloading: "https://dl.fbaipublicfiles.com/fair-esm/regression/esm2_t33_650M_UR50D-contact-regression.pt" to /root/.cache/torch/hub/checkpoints/esm2_t33_650M_UR50D-contact-regression.pt
✅ ESM-2 650M模型加载完成
🚀 ESM-2 650M 蛋白质序列转向量工具

请上传你的蛋白质序列文件（txt格式）:
文件格式: ID<tab>蛋白质ID<tab>序列
--------------------------------------------------


Saving Aindexedtarget.txt to Aindexedtarget.txt

✅ 文件上传成功: Aindexedtarget.txt

📄 处理文件: Aindexedtarget.txt
解析蛋白质序列...
✅ 找到 4294 个蛋白质序列

序列信息:
  0 | P45059 | 长度: 610
  1 | P19113 | 长度: 662
  2 | Q9UI32 | 长度: 602
  ... 还有 4291 个序列

🔄 生成嵌入向量（批次大小: 1）...


生成嵌入向量:  10%|▉         | 410/4294 [01:46<19:05,  3.39it/s]